In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import os
import time

import isx  #Inscopix API; Navigate to C:\Program Files\Inscopix\Data Processing\ ; run pip intsall -e .
import numpy as np

from Python.Helpers import DewanIOhandler

### User Configurables

In [8]:
experiment_name = 'VGAT4-Session1'
data_directory = 'R:\Inscopix_Projects\VGAT4-Session1' # Overwrite if not using local folder
#output_directory = 'D:\VGAT4-Session1\DataAnalysis' # Overwrite if not using local folder

#data_directory = None
output_directory = None

cleanup_interim_files = False

trials = 1
focus_planes = [700]  #If these values do not match the efocus used in the exp, code will crash

#Downsample Settings
spatial_downsample_factor = 4 # Factor of 4 is recommended for CNMFE

#Spatial Bandpass Settings
spatial_bp_low_cutoff = 0.005
spatial_bp_high_cutoff = 0.500

#Motion Correction Settings
max_translation = 20
motion_c_low_bandpass_cutoff = None
motion_c_high_bandpass_cutoff = None

#CNMFE Settings
cell_diameter = 7
merge_threshold = 0.7 #default is 0.7, use default when you have a 10-min video for baseline
gaussian_kernel_size = 3
output_unit_type = 0
minimum_correlation = 0.9
minimum_pnr = 15

### Some processing goes on here

In [9]:
if data_directory is None:
    data_directory = '.\InscopixProcessing\RawData'

if output_directory is None:
    output_directory = '.\InscopixProcessing\DataAnalysis'

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

if not os.path.exists(data_directory):
    os.makedirs(data_directory)

In [10]:
proc_move_files = []
proc_cs_files = []
video_base = DewanIOhandler.get_video_base(data_directory)
video_files = DewanIOhandler.get_video_paths(data_directory)

In [11]:
# print(video_files)

In [6]:
# series_rec_names = {}
#
# for i, focus_plane in enumerate(focus_planes):
#     deinterlaced_files = [file for file in DewanIOhandler.make_isx_path(video_files, output_directory, str(focus_plane))]
#     key_name = f'focal{str(focus_plane)}'
#     series_rec_names[key_name] = deinterlaced_files
#
# series_rec_names_keys = list(series_rec_names.keys())


In [40]:
# Don't think we need any of this anymore. Documentation says deinterleaving is now automatic during preprocessing

#deinterlaced_files = np.reshape(deinterlaced_files_matrix, (1, len(focus_planes) * trials))

# if len(focus_planes) > 1:
# isx.deinterleave(video_files, deinterlaced_files, focus_planes)
# if len(focus_planes) > 1:
#     deinterlaced_files_matrix = np.array((len(focus_planes), trials))
#     for i in range(len(focus_planes)):
#         deinterlaced_files_matrix[i, :] = isx.make_output_file_paths(video_files, output_directory, str(focus_planes(i)))
#     deinterlaced_files = np.reshape(deinterlaced_files_matrix, (1, len(focus_planes) * trials))
#
#     isx.deinterleave(video_files, deinterlaced_files, focus_planes)
#
#     for i in range(len(focus_planes)):
#         keyName = f'focal{focus_planes[i]}'
# else:
#     keyName = f'focal{focus_planes[0]}'
#     series_rec_names[keyName] = video_files
#     series_rec_names_keys = list(series_rec_names.keys())


['R:\\Inscopix_Projects\\VGAT5-Odor-Identity-Expt\\2022-11-16-13-35-25_video.isxd'] 1


In [19]:
# for key in range(len(series_rec_names_keys)):
#     series_name = series_rec_names_keys[key]
#     rec_files = series_rec_names[series_name]
#     first_file = video_files[key]

post_process_files = DewanIOhandler.make_isx_path(video_files, output_directory, 'PP')
if not DewanIOhandler.check_files(post_process_files):
    print('Starting Preprocessing...')
    isx.preprocess(video_files, post_process_files, spatial_downsample_factor=spatial_downsample_factor)

#bandpass_files= [isx.make_output_file_path(post_process_files, output_directory, 'BP')]
bandpass_files= DewanIOhandler.make_isx_path(post_process_files, output_directory, 'BP')

if not DewanIOhandler.check_files(list(bandpass_files)):
    print('Starting Bandpass Filtering...')
    isx.spatial_filter(post_process_files, bandpass_files, low_cutoff=spatial_bp_low_cutoff, high_cutoff=spatial_bp_high_cutoff)

mean_projection_file = os.path.join(output_directory, f'{video_base}-mean_image.isxd')
if not os.path.exists(mean_projection_file):
    print('Creating Mean Image...')
    isx.project_movie(bandpass_files, mean_projection_file, stat_type='mean')

# motion_correction_files = isx.make_output_file_path(bandpass_files, output_directory, 'MC')
motion_correction_files = DewanIOhandler.make_isx_path(bandpass_files, output_directory, 'MC')
# translation_files = isx.make_output_file_path(motion_correction_files, output_directory, 'translations', 'csv')
translation_files = DewanIOhandler.make_isx_path(motion_correction_files, output_directory, 'translations', 'csv')
crop_rect_file = os.path.join(output_directory, f'{video_base}-crop_rect.csv')

if not DewanIOhandler.check_files(motion_correction_files):
    print('Starting Motion Correction...')
    isx.motion_correct(bandpass_files, motion_correction_files, max_translation=max_translation, reference_file_name=mean_projection_file,
                       low_bandpass_cutoff=motion_c_low_bandpass_cutoff, high_bandpass_cutoff=motion_c_high_bandpass_cutoff,
                       output_translation_files=translation_files, output_crop_rect_file=crop_rect_file)

# dff_files = isx.make_output_file_path(motion_correction_files, output_directory, 'DFF')
dff_files = DewanIOhandler.make_isx_path(motion_correction_files, output_directory, 'DFF')

if not DewanIOhandler.check_files(dff_files):
    print('Calculating DF/F...')
    isx.dff(motion_correction_files, dff_files, f0_type='mean')

max_projection_file = DewanIOhandler.make_isx_path(dff_files, output_directory, 'MAX_PROJ')

for i, each in enumerate(max_projection_file):
    if not DewanIOhandler.check_files(max_projection_file):
        start_time = time.strftime("%H:%M:%S")
        print(f"{start_time} Starting Max Projection Creation. This might take a while...")
        isx.project_movie(dff_files[i], each, stat_type='max')
        end_time = time.strftime("%H:%M:%S")
        print(f"{end_time} Max Projection Creation finished. That took a while...")

tiff_path = DewanIOhandler.make_isx_path(max_projection_file, output_directory, addition=None, extention='.tiff')

20:06:45 Starting Max Projection Creation. This might take a while...
20:07:18 Max Projection Creation finished. That took a while...


In [20]:

for i, each in enumerate(tiff_path):
    isx.export_isxd_image_to_tiff(max_projection_file[i], each)

cnmfe_files = DewanIOhandler.make_isx_path(motion_correction_files, output_directory, 'CNMFE')


if not DewanIOhandler.check_files(motion_correction_files) or not DewanIOhandler.check_files(cnmfe_files):
    print('Running CNMFe (This Will Take a Long Time)...')
    start_time = time.time()
    isx.run_cnmfe(motion_correction_files, cnmfe_files, output_directory, cell_diameter=cell_diameter, merge_threshold=merge_threshold, gaussian_kernel_size=gaussian_kernel_size,
                  output_unit_type='df', min_corr=minimum_correlation, min_pnr=minimum_pnr, processing_mode='parallel_patches', num_threads=8)  #  No DF/F on CNMFE Files
    end_time = time.time()
    print(f'Built-in CNMFE Total Elapsed Time: {(end_time - start_time) / 60} (min)')

# movie_filename = f'MC_ForCNMFE{focus_planes[key]}.tiff'
# tiff_movie_file = os.path.join('C:\\Users\\dewanlab.PSY-PDB-C474-B\Desktop', movie_filename)

contour_files = DewanIOhandler.make_isx_path(dff_files, output_directory, 'AC')

if not DewanIOhandler.check_files(dff_files) or not DewanIOhandler.check_files(cnmfe_files) or not DewanIOhandler.check_files(contour_files):
    print('Applying Cell Contours...')
    isx.apply_cell_set(dff_files, cnmfe_files[0], contour_files, 0)

tiff_file_name = f'MC-DFF-AC-TIFF-{video_base}.tiff'
csv_file_name = f'MC-DFF-AC-{video_base}.csv'
json_file_name = f'CONTOUR-{video_base}.json'
props_file_name = f'MC-DFF-AC-{video_base}-Prop.csv'
gpio_input = f'{video_base}.gpio'
gpio_output_filename = f'GPIO-{video_base}.csv'

tiff_image_path = os.path.join(output_directory, tiff_file_name)
trace_file_path = os.path.join(output_directory, csv_file_name)
props_file_path = os.path.join(output_directory, props_file_name)

print('Exporting Traces and Props...')
isx.export_cell_set_to_csv_tiff(contour_files, trace_file_path, tiff_image_path, time_ref='unix', output_props_file= props_file_path)

json_file_path = os.path.join(output_directory, json_file_name)
print('Exporting Cell Contours...')
isx.export_cell_contours(cnmfe_files[0], json_file_path)

gpio_input_path = os.path.join(data_directory, gpio_input)
gpio_output_path = os.path.join(output_directory, gpio_output_filename)

print("Exporting GPIO to csv...")
isx.export_gpio_set_to_csv(gpio_input_path, gpio_output_path, time_ref='unix')

# #Graphing stuff here
# movie_filename = f'DFF_{focus_planes[key]}.mp4'
# movie_file = os.path.join(output_directory, movie_filename)
# print('Exporting Movie to MP4...')
# isx.export_movie_to_mp4(dff_files, movie_file, compression_quality= 0.4, write_invalid_frames= True)
print('Processing Finished!')

if cleanup_interim_files:
    for each in post_process_files, bandpass_files, motion_correction_files, dff_files, cnmfe_files:
        os.remove(each)


Running CNMFe (This Will Take a Long Time)...
Built-in CNMFE Total Elapsed Time: 38.906516130765276 (min)
Applying Cell Contours...
Exporting Traces and Props...
Exporting Cell Contours...
Exporting GPIO to csv...
Processing Finished!
